In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

In [2]:
search = torch.randn(1, 3, 255, 255)
target = torch.randn(1, 3, 127, 127)

In [3]:
IN_CHANNELS = 3

In [60]:
__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152', 'resnext50_32x4d', 'resnext101_32x8d',
           'wide_resnet50_2', 'wide_resnet101_2']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
    'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
    'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
    'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
    'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
}


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=0,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True) # conv1
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0]) # conv2
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0]) # conv3
        self.layer3 = self._make_layer(block, 256, layers[2], stride=1,
                                       dilate=replace_stride_with_dilation[1]) # conv4
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        conv1 = x
        x = self.maxpool(x)

        x = self.layer1(x)
        conv2 = x
        x = self.layer2(x)
        conv3 = x
        x = self.layer3(x)
        conv4 = x
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return conv1, conv2, conv3, conv4

    def forward(self, x):
        return self._forward_impl(x)


In [61]:
def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

In [62]:
model = resnet50()
c1, c2, c3, c4 = model(target)
c11, c22, c33, c44 = model(search)

In [63]:
print(c1.shape, c11.shape)
print(c2.shape, c22.shape)
print(c3.shape, c33.shape)
print(c4.shape, c44.shape)

torch.Size([1, 64, 61, 61]) torch.Size([1, 64, 125, 125])
torch.Size([1, 256, 31, 31]) torch.Size([1, 256, 63, 63])
torch.Size([1, 512, 16, 16]) torch.Size([1, 512, 32, 32])
torch.Size([1, 1024, 16, 16]) torch.Size([1, 1024, 32, 32])


In [137]:
class SearchBone(nn.Module):
    def __init__(self, backbone_channels=1024, out_channels=256):
        super(SearchBone, self).__init__()
        self.backbone_channels = backbone_channels
        self.out_channels = out_channels

        self.backbone = nn.Sequential(*base_layers[0:7])
        self.adjust = nn.Conv2d(backbone_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.backbone(x)
        x = self.adjust(x)
        return x

In [12]:
mm = SearchBone()
mm

SearchBone(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(

In [13]:
x5 = mm(search)
print('x5.shape:', x5.shape)

x5.shape: torch.Size([1, 256, 16, 16])


In [61]:
class TargetBone(nn.Module):
    def __init__(self, backbone_channels=1024, out_channels=256):
        super(TargetBone, self).__init__()
        self.backbone_channels = backbone_channels
        self.out_channels = out_channels

        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(0, 0), bias=False), 
                                   *base_layers[1:3]
                                  )
        self.conv2_x = nn.Sequential(*base_layers[3:5])
        self.conv3_x = nn.Sequential(*base_layers[5:6])
        self.conv4_x = nn.Sequential(*base_layers[6:7])
        self.adjust = nn.Conv2d(backbone_channels, out_channels, kernel_size=1, bias=False)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2_x = self.conv2_x(conv1)
        conv3_x = self.conv3_x(conv2_x)
        conv4_x = self.conv4_x(conv3_x)
        adjust = self.adjust(conv4_x)
        return conv1, conv2_x, conv3_x, conv4_x, adjust

In [62]:
mm = TargetBone()
x1, x2, x3, x4, x5 = mm(target)
print('x1.shape:', x1.shape, 'x2.shape:', x2.shape, 'x3.shape:', x3.shape, 'x4.shape:', x4.shape, 'x5.shape:', x5.shape)

x1.shape: torch.Size([1, 64, 61, 61]) x2.shape: torch.Size([1, 256, 31, 31]) x3.shape: torch.Size([1, 512, 16, 16]) x4.shape: torch.Size([1, 1024, 8, 8]) x5.shape: torch.Size([1, 256, 8, 8])


In [16]:
class PoolModule(nn.Module):
    def __init__(self, kernel_size=(17, 17, 256)):
        super(PoolModule, self).__init__()
        self.maxpool3d = nn.MaxPool3d(kernel_size)

    def forward(self, x):
        x = x.unsqueeze(0)
        x = self.maxpool3d(x)
        x = x.squeeze()
        return x

In [53]:
class ModelHead(nn.Module):
    def __init__(self, ):
        super(ModelHead, self).__init__()
        
        self.search_bone = SearchBone()
        self.target_bone = TargetBone()
    
    def Correlation_func(self, x, kernel):
        x = x.view(1, -1, x.size(2), x.size(3))  # 1 * (b*c) * k * k
        kernel = kernel.view(-1, 1, kernel.size(2), kernel.size(3))  # (b*c) * 1 * H * W
        out = F.conv2d(x, kernel, groups=x.size(1))
        out = out.view(1, x.size(1), out.size(2), out.size(3))
        return out
        

    def forward(self, search, target):
        '''
        TODO: 
            permute for time sequence: (batch, time, channels, input_size, input_size) -->(batch*time, channels, input_size, input_size)
        '''
        search = self.search_bone(search)
        _, _, _, _, target = self.target_bone(target)
        corr_feat = self.Correlation_func(search, target)
        return corr_feat

In [54]:
model = ModelHead()

In [55]:
model(search, target).shape

torch.Size([1, 3, 127, 127])
torch.Size([1, 64, 64, 64])


torch.Size([1, 256, 9, 9])